# SI 330: Data Manipulation 
## 04 - Joining, Combining, and Reshaping

### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

## Learning Objectives

* joining, combining, reshaping...

This lab was inspired by https://pythonhealthcare.org/2018/04/08/32-reshaping-pandas-data-with-stack-unstack-pivot-and-melt/

### IMPORTANT: Replace ```?``` in the following code with your uniqname.

In [602]:
MY_UNIQNAME = '?'

## Before we start...
### <font color="magenta">Q1: (2 points) Please let us know what you found confusing in the last class. </font>
We'll try to take time in the next class to review these concepts next class.


Replace this with your response.

## Review from last class

Recall from last class the ```read_html``` function, which made extracting tables from HTML pages a lot easier than using
BeautifulSoup (in fact, it uses bs4 but hides the ugly details).  Let's warm up for today's class by extracting some information from
a number of Wikipedia pages.

Our top-level goal is to extract information about the _aliases_ of some Lord Of The Rings characters.  Take a look at the Wikipedia page
for [Frodo Baggins](https://en.wikipedia.org/wiki/Frodo_Baggins) to get an idea of the sort of pages we're looking at.

In [603]:
frodo_url = 'https://en.wikipedia.org/wiki/Frodo_Baggins'

In [604]:
frodo_tables = pd.read_html(frodo_url)

In [605]:
frodo_tables[0]

,0,1
0,Frodo Baggins,NaN
1,Aliases,"Mr. Underhill,Maura Labingi"
2,Race,Hobbit
3,Information,NaN
4,Book(s),The Fellowship of the RingThe Two TowersThe Re...


Now let's load the page for [Legolas](https://en.wikipedia.org/wiki/Legolas):

In [606]:
legolas_url = 'https://en.wikipedia.org/wiki/Legolas'
legolas_tables = pd.read_html(legolas_url)

In [607]:
legolas_tables[0]

,0,1
0,NaN,This article relies too much on references to ...


Hmmmm.  That doesn't look quite right.

Let's take a look at some URLs and figure out what's going on:

### <font color="magenta">Q2: (2 points) Inspect the Frodo and Legolas pages and see if you can figure out some _attributes_ of the table we're interested in.  </font>


Describe what you found.

You'll notice that there are some characteristics that the "Information" box share across pages.  We can leverage that 
information by using the ```attrs``` attribute of ```read_html```.  For example, if we wanted to extract  the element(s) that had
an ```id``` of ```info```, we could use

```pd.read_html(url,{'id':'info'})```



### <font color="magenta">Q3: (2 points) Fill in the following code block to extract only the "Information" table for the Legolas page:

In [ ]:
a = {} # create an appropriate dictionary
pd.read_html(legolas_url, attrs=a)

Now let's define a function that, given a Wikipedia URL, will extract the contents of the Aliases component of the infobox table:

In [609]:
def get_aliases(url):
    tables = pd.read_html(url, attrs={'class':'infobox'}) # extract only tables with class=infobox
    print(url,len(tables))   # sanity check: we should have just 1 table
    infotable = tables[0]    # pull the first table into a DataFrame
    ret = ''                 # initialize an empty string for our return value
    try:                     # in case the next line throws an exception
        x = infotable.set_index(0).loc['Aliases'] # setting the index on column 0 will allow us to use .loc to look up the value of 'Aliases'
        ret = x.values[0]
    except:
        ret = 'None'
    return ret

And let's try it out:

In [ ]:
get_aliases(legolas_url)

So far, so good.  It seems to work.  Now let's set up a DataFrame with a bunch of URLs:

In [611]:
urls = ['https://en.wikipedia.org/wiki/Gimli_(Middle-earth)',
        'https://en.wikipedia.org/wiki/Frodo_Baggins',
        'https://en.wikipedia.org/wiki/Legolas',
        'https://en.wikipedia.org/wiki/Bilbo_Baggins',
        'https://en.wikipedia.org/wiki/Samwise_Gamgee',
        'https://en.wikipedia.org/wiki/Peregrin_Took',
        'https://en.wikipedia.org/wiki/Boromir',
        'https://en.wikipedia.org/wiki/Galadriel',
        'https://en.wikipedia.org/wiki/Meriadoc_Brandybuck']
names = ['Gimli',
         'Frodo',
         'Legolas',
         'Bilbo',
         'Sam',
         'Pippin',
         'Boromir',
         'Galadriel',
         'Meriadoc']

In [612]:
udf = pd.DataFrame()
udf['name'] = names
udf['url'] = urls

In [614]:
udf

,name,url
0,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
1,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
2,Legolas,https://en.wikipedia.org/wiki/Legolas
3,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
4,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee
5,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
6,Boromir,https://en.wikipedia.org/wiki/Boromir
7,Galadriel,https://en.wikipedia.org/wiki/Galadriel
8,Meriadoc,https://en.wikipedia.org/wiki/Meriadoc_Brandybuck


The pythonic way of iterating through each of those rows would involve the use of some sort of ```for``` loop.  In pandas,
however, as can use the ```apply``` function to process an entire column!

In [ ]:
udf['url'].apply(get_aliases)

We can take the resulting Series and assign it to a new column in our DataFrame:

In [ ]:
udf['aliases'] = udf['url'].apply(get_aliases)

In [ ]:
udf

Let's just put the ```udf``` DataFrame aside for now.  We'll return to it later.

## Creating DataFrames and Exploring Indexes

Let's load the usual libraries...

In [616]:
import pandas as pd
import numpy as np

Let's create some lists of data that we can use to construct a DataFrame:

In [617]:
names = ['Gandalf',
         'Gimli',
         'Frodo',
         'Legolas',
         'Bilbo',
         'Sam',
         'Pippin',
         'Boromir',
         'Aragorn',
         'Galadriel',
         'Meriadoc',
        'Lily']
races = ['Maia',
         'Dwarf',
         'Hobbit',
         'Elf',
         'Hobbit',
         'Hobbit',
         'Hobbit',
         'Man',
         'Man',
         'Elf',
         'Hobbit',
        'Hobbit']
magic = [10, 1, 4, 6, 4, 2, 0, 0, 2, 9, 0, np.NaN]
aggression = [7, 10, 2, 5, 1, 6, 3, 8, 7, 2, 4, np.NaN ]
stealth = [8, 2, 5, 10, 5, 4 ,5, 3, 9, 10, 6, np.NaN]

There are a few different ways to construct a DataFrame.  We can either use an empty constructor and assign Series:

In [618]:
df = pd.DataFrame()

df['name'] = names
df['race'] = races
df['magic_power'] = magic
df['aggression'] = aggression
df['stealth'] = stealth

In [619]:
df

,name,race,magic_power,aggression,stealth
0,Gandalf,Maia,10.0,7.0,8.0
1,Gimli,Dwarf,1.0,10.0,2.0
2,Frodo,Hobbit,4.0,2.0,5.0
3,Legolas,Elf,6.0,5.0,10.0
4,Bilbo,Hobbit,4.0,1.0,5.0
5,Sam,Hobbit,2.0,6.0,4.0
6,Pippin,Hobbit,0.0,3.0,5.0
7,Boromir,Man,0.0,8.0,3.0
8,Aragorn,Man,2.0,7.0,9.0
9,Galadriel,Elf,9.0,2.0,10.0


Alternatively, we could have set things up with a dict:

In [620]:
df = pd.DataFrame({'name': names,'race':races,'magic':magic,'aggression': aggression,'stealth':stealth})

In [621]:
df

,name,race,magic,aggression,stealth
0,Gandalf,Maia,10.0,7.0,8.0
1,Gimli,Dwarf,1.0,10.0,2.0
2,Frodo,Hobbit,4.0,2.0,5.0
3,Legolas,Elf,6.0,5.0,10.0
4,Bilbo,Hobbit,4.0,1.0,5.0
5,Sam,Hobbit,2.0,6.0,4.0
6,Pippin,Hobbit,0.0,3.0,5.0
7,Boromir,Man,0.0,8.0,3.0
8,Aragorn,Man,2.0,7.0,9.0
9,Galadriel,Elf,9.0,2.0,10.0


Let's take a look at the index on the resulting DataFrame:

In [622]:
df.index

RangeIndex(start=0, stop=12, step=1)

We can set the index to something more useful than the default RangeIndex:

In [623]:
df_nameindexed = df.set_index('name')

And if we take a look at the results, we see that we have a pandas Index instead of a RangeIndex:

In [624]:
df_nameindexed.index

Index(['Gandalf', 'Gimli', 'Frodo', 'Legolas', 'Bilbo', 'Sam', 'Pippin',
       'Boromir', 'Aragorn', 'Galadriel', 'Meriadoc', 'Lily'],
      dtype='object', name='name')

In [625]:
df_nameindexed

,race,magic,aggression,stealth
name,,,,
Gandalf,Maia,10.0,7.0,8.0
Gimli,Dwarf,1.0,10.0,2.0
Frodo,Hobbit,4.0,2.0,5.0
Legolas,Elf,6.0,5.0,10.0
Bilbo,Hobbit,4.0,1.0,5.0
Sam,Hobbit,2.0,6.0,4.0
Pippin,Hobbit,0.0,3.0,5.0
Boromir,Man,0.0,8.0,3.0
Aragorn,Man,2.0,7.0,9.0


Setting the name Series as the index allows us to do things like:

In [626]:
df_nameindexed.loc['Aragorn']

race          Man
magic           2
aggression      7
stealth         9
Name: Aragorn, dtype: object

Now recall the Hierarchical indexing from the readings.  We can pass a list of column names to set_index to create a Hierarchical Index:

In [627]:
df_racename_indexed = df.set_index(['race','name'])

In [628]:
df_racename_indexed.index

MultiIndex(levels=[['Dwarf', 'Elf', 'Hobbit', 'Maia', 'Man'], ['Aragorn', 'Bilbo', 'Boromir', 'Frodo', 'Galadriel', 'Gandalf', 'Gimli', 'Legolas', 'Lily', 'Meriadoc', 'Pippin', 'Sam']],
           labels=[[3, 0, 2, 1, 2, 2, 2, 4, 4, 1, 2, 2], [5, 6, 3, 7, 1, 11, 10, 2, 0, 4, 9, 8]],
           names=['race', 'name'])

This will allow us to get a DataFrame that matches a value on the outer index:

In [629]:
df_racename_indexed.loc['Hobbit']

,magic,aggression,stealth
name,,,
Frodo,4.0,2.0,5.0
Bilbo,4.0,1.0,5.0
Sam,2.0,6.0,4.0
Pippin,0.0,3.0,5.0
Meriadoc,0.0,4.0,6.0
Lily,NaN,NaN,NaN


We can also use the index on a Series to match the outer index:

In [630]:
df_racename_indexed['magic'].loc['Hobbit']

name
Frodo       4.0
Bilbo       4.0
Sam         2.0
Pippin      0.0
Meriadoc    0.0
Lily        NaN
Name: magic, dtype: float64

Or both indexes:

In [631]:
df_racename_indexed['magic'].loc['Hobbit','Frodo']

4.0

Or just the inner index:

In [632]:
df_racename_indexed['magic'].loc[:,'Frodo']

race
Hobbit    4.0
Name: magic, dtype: float64

## Stacking and Unstacking

Stacking takes "wide" data and makes it "taller"

In [633]:
df.set_index(['race']).stack()

race              
Maia    name            Gandalf
        magic                10
        aggression            7
        stealth               8
Dwarf   name              Gimli
        magic                 1
        aggression           10
        stealth               2
Hobbit  name              Frodo
        magic                 4
        aggression            2
        stealth               5
Elf     name            Legolas
        magic                 6
        aggression            5
        stealth              10
Hobbit  name              Bilbo
        magic                 4
        aggression            1
        stealth               5
        name                Sam
        magic                 2
        aggression            6
        stealth               4
        name             Pippin
        magic                 0
        aggression            3
        stealth               5
Man     name            Boromir
        magic                 0
        aggression   

If we call reset_index on the resulting Series, we get the following DataFrame:

In [634]:
df.set_index(['race']).stack().reset_index()

,race,level_1,0
0,Maia,name,Gandalf
1,Maia,magic,10
2,Maia,aggression,7
3,Maia,stealth,8
4,Dwarf,name,Gimli
5,Dwarf,magic,1
6,Dwarf,aggression,10
7,Dwarf,stealth,2
8,Hobbit,name,Frodo
9,Hobbit,magic,4


The column names in the above DataFrame aren't particularly helpful, so we can rename them:

In [635]:
df.set_index(['race']).stack().reset_index().rename(columns = {'level_0':'ID','level_1':'variable',0:'value'})

,race,variable,value
0,Maia,name,Gandalf
1,Maia,magic,10
2,Maia,aggression,7
3,Maia,stealth,8
4,Dwarf,name,Gimli
5,Dwarf,magic,1
6,Dwarf,aggression,10
7,Dwarf,stealth,2
8,Hobbit,name,Frodo
9,Hobbit,magic,4


You can do the opposite of stacking by using the ```unstack``` function:

In [636]:
df_stacked = df.stack()

In [637]:
df_stacked

0   name            Gandalf
    race               Maia
    magic                10
    aggression            7
    stealth               8
1   name              Gimli
    race              Dwarf
    magic                 1
    aggression           10
    stealth               2
2   name              Frodo
    race             Hobbit
    magic                 4
    aggression            2
    stealth               5
3   name            Legolas
    race                Elf
    magic                 6
    aggression            5
    stealth              10
4   name              Bilbo
    race             Hobbit
    magic                 4
    aggression            1
    stealth               5
5   name                Sam
    race             Hobbit
    magic                 2
    aggression            6
    stealth               4
6   name             Pippin
    race             Hobbit
    magic                 0
    aggression            3
    stealth               5
7   name            

In [638]:
df_stacked.unstack()

,name,race,magic,aggression,stealth
0,Gandalf,Maia,10,7,8
1,Gimli,Dwarf,1,10,2
2,Frodo,Hobbit,4,2,5
3,Legolas,Elf,6,5,10
4,Bilbo,Hobbit,4,1,5
5,Sam,Hobbit,2,6,4
6,Pippin,Hobbit,0,3,5
7,Boromir,Man,0,8,3
8,Aragorn,Man,2,7,9
9,Galadriel,Elf,9,2,10


Why would we want to stack or unstack?  It depends on what sorts of analyses we want to do "downstream".  It's also the basis for pivoting, melting, and pivot tables, which we'll cover in the next class.

## Joining Data



Let's say we have another CSV file that contains URLs to Wikipedia pages for some of the LOTR characters:

In [639]:
urls = pd.read_csv('data/lotr_wikipedia.csv')

In [640]:
urls

,name,url
0,Gandalf,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,https://en.wikipedia.org/wiki/Galadriel


Let's take a look at the original DataFrame:

In [641]:
df

,name,race,magic,aggression,stealth
0,Gandalf,Maia,10.0,7.0,8.0
1,Gimli,Dwarf,1.0,10.0,2.0
2,Frodo,Hobbit,4.0,2.0,5.0
3,Legolas,Elf,6.0,5.0,10.0
4,Bilbo,Hobbit,4.0,1.0,5.0
5,Sam,Hobbit,2.0,6.0,4.0
6,Pippin,Hobbit,0.0,3.0,5.0
7,Boromir,Man,0.0,8.0,3.0
8,Aragorn,Man,2.0,7.0,9.0
9,Galadriel,Elf,9.0,2.0,10.0


It looks like the rows are "aligned", so we can use the ```concat``` function to concatenate the two DataFrames.
Note that we specify the axis to be the columns.  The default is to concatenate by rows, which isn't what we want.

In [642]:
pd.concat([df,urls],axis="columns")

,name,race,magic,aggression,stealth,name,url
0,Gandalf,Maia,10.0,7.0,8.0,Gandalf,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,Legolas,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,Boromir,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,Aragorn,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,Galadriel,https://en.wikipedia.org/wiki/Galadriel


That's great, and it's consistent with what we've used in previous classes.  But what happens if the 
rows in the two DataFrames don't match up?  Let's load another file that has a slightly different
sequence of rows:

In [643]:
urls_wrong_order = pd.read_csv('data/lotr_wikipedia_wrong_order.csv')

In [644]:
urls_wrong_order

,name,url
0,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
1,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
2,Legolas,https://en.wikipedia.org/wiki/Legolas
3,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
4,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee
5,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
6,Boromir,https://en.wikipedia.org/wiki/Boromir
7,Aragorn,https://en.wikipedia.org/wiki/Aragorn
8,Galadriel,https://en.wikipedia.org/wiki/Galadriel
9,Meriadoc,https://en.wikipedia.org/wiki/Meriadoc_Brandybuck


In [645]:
pd.concat([df,urls_wrong_order],axis="columns")

,name,race,magic,aggression,stealth,name,url
0,Gandalf,Maia,10.0,7.0,8.0,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
1,Gimli,Dwarf,1.0,10.0,2.0,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
2,Frodo,Hobbit,4.0,2.0,5.0,Legolas,https://en.wikipedia.org/wiki/Legolas
3,Legolas,Elf,6.0,5.0,10.0,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
4,Bilbo,Hobbit,4.0,1.0,5.0,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee
5,Sam,Hobbit,2.0,6.0,4.0,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
6,Pippin,Hobbit,0.0,3.0,5.0,Boromir,https://en.wikipedia.org/wiki/Boromir
7,Boromir,Man,0.0,8.0,3.0,Aragorn,https://en.wikipedia.org/wiki/Aragorn
8,Aragorn,Man,2.0,7.0,9.0,Galadriel,https://en.wikipedia.org/wiki/Galadriel
9,Galadriel,Elf,9.0,2.0,10.0,Meriadoc,https://en.wikipedia.org/wiki/Meriadoc_Brandybuck


Take a closer look at the name and url columns.  Something's not quite right.

We can work around that by using the appropriate indexing and then using the SQL-like ```merge``` function.

In [646]:
df_names = df.set_index('name')

In [647]:
df_names

,race,magic,aggression,stealth
name,,,,
Gandalf,Maia,10.0,7.0,8.0
Gimli,Dwarf,1.0,10.0,2.0
Frodo,Hobbit,4.0,2.0,5.0
Legolas,Elf,6.0,5.0,10.0
Bilbo,Hobbit,4.0,1.0,5.0
Sam,Hobbit,2.0,6.0,4.0
Pippin,Hobbit,0.0,3.0,5.0
Boromir,Man,0.0,8.0,3.0
Aragorn,Man,2.0,7.0,9.0


In [648]:
urls_wrong_order_names = urls_wrong_order.set_index('name')

In [649]:
df_names.join(urls_wrong_order_names)

,race,magic,aggression,stealth,url
name,,,,,
Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn


In [650]:
df.head()

,name,race,magic,aggression,stealth
0,Gandalf,Maia,10.0,7.0,8.0
1,Gimli,Dwarf,1.0,10.0,2.0
2,Frodo,Hobbit,4.0,2.0,5.0
3,Legolas,Elf,6.0,5.0,10.0
4,Bilbo,Hobbit,4.0,1.0,5.0


In [651]:
urls_wrong_order.head()

,name,url
0,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
1,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
2,Legolas,https://en.wikipedia.org/wiki/Legolas
3,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
4,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee


In [652]:
urls_wrong_order['name']

0         Gimli
1         Frodo
2       Legolas
3         Bilbo
4           Sam
5        Pippin
6       Boromir
7       Aragorn
8     Galadriel
9      Meriadoc
10      Gandalf
Name: name, dtype: object

In [653]:
df['name']

0       Gandalf
1         Gimli
2         Frodo
3       Legolas
4         Bilbo
5           Sam
6        Pippin
7       Boromir
8       Aragorn
9     Galadriel
10     Meriadoc
11         Lily
Name: name, dtype: object

In [654]:
df.merge(urls_wrong_order,on='name')

,name,race,magic,aggression,stealth,url
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel


Now let's add a few additional URLs:

In [655]:
urls_extras = pd.read_csv("data/lotr_wikipedia_extras.csv")

In [656]:
urls_extras

,name,url
0,Treebeard,https://en.wikipedia.org/wiki/Treebeard
1,Elrond,https://en.wikipedia.org/wiki/Elrond


And now let's use concat to add the new entries to the DataFrame.

In [657]:
urls_complete = pd.concat([urls,urls_extras])

In [658]:
urls_complete

,name,url
0,Gandalf,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,https://en.wikipedia.org/wiki/Galadriel


Now that we've got a complete (for our purposes) list of URLs, let's use that DataFrame and our original
one to demonstrate the different types of ```join```s.

By default, ```join``` uses a left join, which means the all the values from the "left"
side are used, whether or not there's a corresponding entry from the "right" side.  In the example 
below, note that the url value for "Lily" is "NaN":

In [659]:
df.merge(urls_wrong_order,on='name',how='left')

,name,race,magic,aggression,stealth,url
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel


The "opposite" of a left join is, perhaps unsurprisingly, a "right" join, in which
all the values from the "right" side are used, whether or not a corresponding
value from the "left" side exists. Note in the following example that "Lily" has
disappeared, and Treebeard and Elrond lack information about "race", "magic", "aggression", and "stealth".

In [660]:
df.merge(urls_wrong_order,on='name',how='right')

,name,race,magic,aggression,stealth,url
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel


In addition to "left" and "right" joins, we have "outer" joins, which include
values from both the "left" and "right" DataFrames, regardless of whether
there are corresponding values in the other DataFrame.  Note that all of 
"Lily", "Treebeard" and "Elrond" are present in the following DataFrame:

In [661]:
df.merge(urls_wrong_order,on='name',how='outer')

,name,race,magic,aggression,stealth,url
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel


Finally, there are "inner" joins, which include only those values that exist in both the "left" and "right" DataFrames:

In [662]:
df.merge(urls_wrong_order,on='name',how='inner')

,name,race,magic,aggression,stealth,url
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel


Sometimes it's nice to know how a particular row got added to the resulting DataFrame.  Using ```indicator=True```
allows us to examine this:

In [663]:
df.merge(urls_complete,how='outer',indicator=True)

,name,race,magic,aggression,stealth,url,_merge
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf,both
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...,both
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins,both
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas,both
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins,both
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee,both
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took,both
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir,both
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn,both
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel,both


You'll note that we used the ```merge``` function from the DataFrame and passed in the other DataFrame as an argument.
You can also call the ```merge``` function from pandas directly and pass it the two DataFrames you are merging:

In [664]:
pd.merge(df,urls_complete,how='outer',indicator=True)

,name,race,magic,aggression,stealth,url,_merge
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf,both
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...,both
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins,both
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas,both
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins,both
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee,both
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took,both
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir,both
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn,both
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel,both


### <font color="magenta">Q4: (4 points) Join the ```udf``` DataFrame (that contains aliases) to the ```df``` DataFrame using an appropriate merge

In [665]:
# Insert your code here

# END OF NOTEBOOK
Please remember to submit your notebook in .ipynb and .html formats.